# 6-DOF Quadrotor Reachability Analysis

This notebook demonstrates how to use the Hamilton-Jacobi reachability analysis for a 6-DOF quadrotor system using differentially flat outputs.

We'll analyze:
1. Safe landing regions
2. Reachable sets for position control
3. Safe flight corridors

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

import hj_reachability as hj
from hj_reachability.systems.quadrotor_6dof import Quadrotor6DOF

## 1. System Setup

First, let's create our quadrotor system and define the computational grid.

In [ ]:
# Create quadrotor system
quad = Quadrotor6DOF(
    max_velocity=5.0,        # m/s
    max_acceleration=5.0,    # m/s^2
    max_yaw_rate=3.14,      # rad/s
    max_yaw_accel=3.14,     # rad/s^2
    control_mode="min"      # minimize time to reach target
)

# Define grid parameters
# Note: We use a coarse grid for demonstration. Increase resolution for better results.
grid = hj.Grid.from_lattice_parameters_and_boundary_conditions(
    domain=hj.sets.Box(
        lo=np.array([-5., -5., -5., -5., -5., -5., -np.pi, -3.14]),  # [x,y,z,vx,vy,vz,psi,psi_dot]
        hi=np.array([5., 5., 5., 5., 5., 5., np.pi, 3.14])
    ),
    shape=(15, 15, 15, 11, 11, 11, 8, 8),  # Grid resolution
    periodic_dims=6  # yaw is periodic
)

## 2. Safe Landing Analysis

Let's compute the backward reachable tube (BRT) for safe landing conditions.
We define "safe landing" as reaching a small region near the origin with near-zero velocity.

In [ ]:
def create_landing_target(grid, position_radius=0.5, velocity_threshold=0.5):
    """Create target set for safe landing."""
    # Distance from origin in position space
    position_distance = jnp.linalg.norm(grid.states[..., :3], axis=-1)
    
    # Velocity magnitude
    velocity_magnitude = jnp.linalg.norm(grid.states[..., 3:6], axis=-1)
    
    # Combine conditions: inside position radius AND low velocity
    return jnp.maximum(position_distance - position_radius,
                      velocity_magnitude - velocity_threshold)

# Create initial value function (negative inside target set)
initial_values = create_landing_target(grid)

# Configure solver
solver_settings = hj.SolverSettings.with_accuracy(
    "medium",
    hamiltonian_postprocessor=hj.solver.backwards_reachable_tube
)

# Solve backward reachability
times = jnp.linspace(0, -2.0, 41)  # 2 second time horizon
values = hj.solve(solver_settings, quad, grid, times, initial_values)

## 3. Visualize Results

Let's visualize slices of the reachable set at different velocities and yaw angles.

In [ ]:
def plot_position_slice(values, grid, time_index=-1, vx_index=5, vy_index=5, vz_index=5, psi_index=4):
    """Plot a 3D slice of the value function in position space."""
    # Extract the slice
    slice_values = values[time_index, :, :, :, vx_index, vy_index, vz_index, psi_index]
    
    # Create the 3D visualization
    fig = go.Figure(data=go.Isosurface(
        x=grid.states[..., 0].reshape(-1),
        y=grid.states[..., 1].reshape(-1),
        z=grid.states[..., 2].reshape(-1),
        value=slice_values.reshape(-1),
        isomin=0,
        isomax=0,
        surface_count=1,
        colorscale='viridis',
        caps=dict(x_show=False, y_show=False, z_show=False)
    ))
    
    # Update layout
    fig.update_layout(
        scene=dict(
            xaxis_title='X (m)',
            yaxis_title='Y (m)',
            zaxis_title='Z (m)'
        ),
        title=f'Reachable Set at t={times[time_index]:.1f}s'
    )
    
    return fig

# Plot the results
fig = plot_position_slice(values, grid)
fig.show()

## 4. Optimal Control Extraction

We can extract the optimal control policy from the value function gradient.

In [ ]:
def get_optimal_control_at_state(quad, grid, values, state, time_index=-1):
    """Compute optimal control at a given state."""
    # Get value function gradient at the state
    grad_values = grid.grad_values(values[time_index])
    
    # Interpolate gradient at the state
    costate = grid.interpolate(grad_values, state)
    
    # Compute optimal control
    return quad.get_optimal_control(state, costate)

# Example: Compute optimal control at a specific state
example_state = jnp.array([2.0, 1.0, 1.0, 0.5, 0.5, 0.5, 0.0, 0.0])
optimal_control = get_optimal_control_at_state(quad, grid, values, example_state)
print("Optimal control at example state:")
print(f"ax = {optimal_control[0]:.2f} m/s²")
print(f"ay = {optimal_control[1]:.2f} m/s²")
print(f"az = {optimal_control[2]:.2f} m/s²")
print(f"psi_ddot = {optimal_control[3]:.2f} rad/s²")

## 5. Analysis

The visualization shows the set of initial states from which the quadrotor can safely reach the landing target within the given time horizon. Points inside the zero level set (surface shown) can reach the target safely, while points outside cannot guarantee safe landing within the time horizon.

Key observations:
1. The reachable set shape is influenced by the maximum velocity and acceleration constraints
2. The set is symmetric due to the symmetric control constraints
3. The optimal control values show how to guide the quadrotor toward the target

This analysis can be used for:
- Safe trajectory planning
- Emergency landing verification
- Control envelope analysis
- Safety verification